# Multilayer perceptron

This software uses cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline 

See MIT License in 
https://github.com/cauchyturing/UCR_Time_Series_Classification_Deep_Learning_Baseline/README.md

Wang, Z., Yan, W. and Oates, T. (2017) ‘Time series classification from scratch with deep neural networks: A strong baseline’, 2017 International Joint Conference on Neural Networks (IJCNN), pp. 1578–1585 [Online.](https://arxiv.org/abs/1611.06455 "Wang et al. (2017)")

## Data
Additional datasets are available at http://www.timeseriesclassification.com/. Save additional datasets in deepscent/data. E.g. deepscent/data/Adiac/Adiac_TRAIN.txt

## Formatting
Left align the tables in this Notebook.

In [12]:
%%html
<style>
  table {margin-left: 0 !important;}
</style>

## Train MLP
Expected running time for various datasets, on a single NVIDIA GeForce GTX 1080 Ti Graphics Card -

Dataset   | Training time
:-------  | :--------
Adiac     | 18 minutes
GunPoint  | 11 minutes


In [13]:
import tensorflow as tf
import tensorflow.keras as keras

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras import utils
import numpy as np
from tensorflow.keras.callbacks import ReduceLROnPlateau
import pandas as pd

import os
import time
from datetime import datetime
import pathlib

np.random.seed(813306)

      
def readucr(filename):
    data = np.loadtxt(filename)
    Y = data[:,0]
    X = data[:,1:]
    return X, Y
  
nb_epochs = 5000

# Directories
fdir = '../data'  
logs_dir = '../logs'
timestamp = '{:%Y-%m-%dT%H:%M}'.format(datetime.now())
logs_dir = logs_dir +'/' + timestamp
#flist = ['Adiac', 'Beef', 'CBF', 'ChlorineConcentration', 'CinC_ECG_torso', 'Coffee', 'Cricket_X', 'Cricket_Y', 'Cricket_Z', 
#'DiatomSizeReduction', 'ECGFiveDays', 'FaceAll', 'FaceFour', 'FacesUCR', '50words', 'FISH', 'GunPoint', 'Haptics', 
#'InlineSkate', 'ItalyPowerDemand', 'Lighting2', 'Lighting7', 'MALLAT', 'MedicalImages', 'MoteStrain', 'NonInvasiveFatalECG_Thorax1', 
#'NonInvasiveFatalECG_Thorax2', 'OliveOil', 'OSULeaf', 'SonyAIBORobotSurface', 'SonyAIBORobotSurfaceII', 'StarLightCurves', 'SwedishLeaf', 'Symbols', 
#'synthetic_control', 'Trace', 'TwoLeadECG', 'Two_Patterns', 'uWaveGestureLibrary_X', 'uWaveGestureLibrary_Y', 'uWaveGestureLibrary_Z', 'wafer', 'WordsSynonyms', 'yoga']

flist = ['Adiac']
for each in flist:
    fname = each
    print('Running dataset', fname)
    x_train, y_train = readucr(fdir+'/'+fname+'/'+fname+'_TRAIN.txt')
    x_test, y_test = readucr(fdir+'/'+fname+'/'+fname+'_TEST.txt')
    nb_classes =len(np.unique(y_test))
    y_train = (y_train - y_train.min())/(y_train.max()-y_train.min())*(nb_classes-1)
    y_test = (y_test - y_test.min())/(y_test.max()-y_test.min())*(nb_classes-1)
    batch_size = int(min(x_train.shape[0]/10, 16))
    
    Y_train = utils.to_categorical(y_train, nb_classes)
    Y_test = utils.to_categorical(y_test, nb_classes)
     
    x_train_mean = x_train.mean()
    x_train_std = x_train.std()
    x_train = (x_train - x_train_mean)/(x_train_std)
     
   # x_test_min = np.min(x_test, axis = 1, keepdims=1)
   # x_test_max = np.max(x_test, axis = 1, keepdims=1)
    x_test = (x_test - x_train_mean)/(x_train_std)
     
    #x_train = x_train.reshape(x_train.shape + (1,))
    #x_test = x_test.reshape(x_test.shape + (1,))
    
    x = Input(x_train.shape[1:])
    y= Dropout(0.1)(x)
    y = Dense(500, activation='relu')(x)
    y = Dropout(0.2)(y)
    y = Dense(500, activation='relu')(y)
    y = Dropout(0.2)(y)
    y = Dense(500, activation = 'relu')(y)
    y = Dropout(0.3)(y)
    out = Dense(nb_classes, activation='softmax')(y)
     
    model = Model(x, out)
     
    optimizer = keras.optimizers.Adadelta(rho=0.95, epsilon=1e-8)    
    model.compile(loss='categorical_crossentropy',
                  optimizer=optimizer,
                  metrics=['accuracy'])
     
    reduce_lr = ReduceLROnPlateau(monitor = 'loss', factor=0.5,
                      patience=200, min_lr=0.1)
    

    start = time.time()
    hist = model.fit(x_train, Y_train, batch_size=batch_size, epochs=nb_epochs,
              verbose=1, validation_data=(x_test, Y_test), 
                #callbacks = [TestCallback((x_train, Y_train)), reduce_lr, keras.callbacks.TensorBoard(log_dir='./log'+fname, histogram_freq=1)])
                 callbacks=[reduce_lr])
    end = time.time()
    
    log = pd.DataFrame(hist.history)
    # Print results. Print the testing results which has the lowest training loss.
    print('Training complete on', fname)
    print('Training time ', end-start, 'seconds, which is about', round((end-start)/60), 'minutes.')
    print('Test result with the lowest training loss. Loss and accuracy are -')

    loss = log.loc[log['loss'].idxmin]['loss']
    val_acc = log.loc[log['loss'].idxmin]['val_acc']
    print(loss, val_acc)
    print('For the results table -')
    print('|#   |',loss,'  |',val_acc,' |')
    # Save log file.
    pathlib.Path(logs_dir+'/'+fname).mkdir(parents=True, exist_ok=True) 
    print('Saving logs to',logs_dir+'/'+fname+'/history.csv')
    log.to_csv(logs_dir+'/'+fname+'/history.csv')
 



Running dataset GunPoint
Train on 50 samples, validate on 150 samples
Epoch 1/5000
50/50 [==============================] - 1s 10ms/step - loss: 0.6761 - acc: 0.6000 - val_loss: 0.5438 - val_acc: 0.6933
Epoch 2/5000
50/50 [==============================] - 0s 2ms/step - loss: 0.4910 - acc: 0.7800 - val_loss: 0.4769 - val_acc: 0.7333
Epoch 3/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.4810 - acc: 0.7600 - val_loss: 0.4263 - val_acc: 0.7333
Epoch 4/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.4060 - acc: 0.8400 - val_loss: 0.6150 - val_acc: 0.7067
Epoch 5/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.3289 - acc: 0.8200 - val_loss: 0.3863 - val_acc: 0.8133
Epoch 6/5000
50/50 [==============================] - 0s 2ms/step - loss: 0.4196 - acc: 0.7200 - val_loss: 0.4563 - val_acc: 0.7333
Epoch 7/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.2389 - acc: 0.9000 - val_loss: 0.4333 - val_acc: 0.7867
Epoch

50/50 [==============================] - 0s 3ms/step - loss: 0.0162 - acc: 1.0000 - val_loss: 0.1640 - val_acc: 0.9333
Epoch 63/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0162 - acc: 1.0000 - val_loss: 0.1600 - val_acc: 0.9333
Epoch 64/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0074 - acc: 1.0000 - val_loss: 0.1690 - val_acc: 0.9200
Epoch 65/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0132 - acc: 1.0000 - val_loss: 0.1822 - val_acc: 0.9400
Epoch 66/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0089 - acc: 1.0000 - val_loss: 0.2137 - val_acc: 0.9200
Epoch 67/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0116 - acc: 1.0000 - val_loss: 0.2558 - val_acc: 0.9200
Epoch 68/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.1668 - acc: 0.9200 - val_loss: 0.1672 - val_acc: 0.9533
Epoch 69/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0102 -

50/50 [==============================] - 0s 3ms/step - loss: 0.0057 - acc: 1.0000 - val_loss: 0.2487 - val_acc: 0.9467
Epoch 124/5000
50/50 [==============================] - 0s 2ms/step - loss: 8.3034e-04 - acc: 1.0000 - val_loss: 0.2645 - val_acc: 0.9400
Epoch 125/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0347 - acc: 0.9800 - val_loss: 0.4803 - val_acc: 0.8733
Epoch 126/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0518 - acc: 0.9600 - val_loss: 0.2672 - val_acc: 0.9467
Epoch 127/5000
50/50 [==============================] - 0s 2ms/step - loss: 4.5591e-04 - acc: 1.0000 - val_loss: 0.2706 - val_acc: 0.9467
Epoch 128/5000
50/50 [==============================] - 0s 3ms/step - loss: 4.2156e-04 - acc: 1.0000 - val_loss: 0.2683 - val_acc: 0.9467
Epoch 129/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0032 - acc: 1.0000 - val_loss: 0.2901 - val_acc: 0.9333
Epoch 130/5000
50/50 [==============================] - 0s 3ms/st

Epoch 183/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.4234e-05 - acc: 1.0000 - val_loss: 0.2651 - val_acc: 0.9467
Epoch 184/5000
50/50 [==============================] - 0s 3ms/step - loss: 8.9550e-05 - acc: 1.0000 - val_loss: 0.3530 - val_acc: 0.9267
Epoch 185/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0133 - acc: 1.0000 - val_loss: 0.5908 - val_acc: 0.9000
Epoch 186/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0015 - acc: 1.0000 - val_loss: 0.2727 - val_acc: 0.9467
Epoch 187/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.1871e-04 - acc: 1.0000 - val_loss: 0.2918 - val_acc: 0.9333
Epoch 188/5000
50/50 [==============================] - 0s 3ms/step - loss: 8.1315e-05 - acc: 1.0000 - val_loss: 0.2872 - val_acc: 0.9467
Epoch 189/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.0773e-04 - acc: 1.0000 - val_loss: 0.2836 - val_acc: 0.9467
Epoch 190/5000
50/50 [====================

50/50 [==============================] - 0s 2ms/step - loss: 3.8089e-05 - acc: 1.0000 - val_loss: 0.3334 - val_acc: 0.9533
Epoch 243/5000
50/50 [==============================] - 0s 3ms/step - loss: 9.6875e-05 - acc: 1.0000 - val_loss: 0.3896 - val_acc: 0.9400
Epoch 244/5000
50/50 [==============================] - 0s 3ms/step - loss: 5.6359e-05 - acc: 1.0000 - val_loss: 0.4208 - val_acc: 0.9200
Epoch 245/5000
50/50 [==============================] - 0s 3ms/step - loss: 3.0132e-04 - acc: 1.0000 - val_loss: 0.5345 - val_acc: 0.9067
Epoch 246/5000
50/50 [==============================] - 0s 3ms/step - loss: 0.0057 - acc: 1.0000 - val_loss: 0.3863 - val_acc: 0.9333
Epoch 247/5000
50/50 [==============================] - 0s 3ms/step - loss: 7.4584e-05 - acc: 1.0000 - val_loss: 0.3717 - val_acc: 0.9467
Epoch 248/5000
50/50 [==============================] - 0s 3ms/step - loss: 7.0814e-05 - acc: 1.0000 - val_loss: 0.3726 - val_acc: 0.9400
Epoch 249/5000
50/50 [==============================]

...

Epoch 4726/5000
50/50 [==============================] - 0s 3ms/step - loss: 4.2915e-07 - acc: 1.0000 - val_loss: 0.3910 - val_acc: 0.9333
Epoch 4727/5000
50/50 [==============================] - 0s 2ms/step - loss: 4.3631e-07 - acc: 1.0000 - val_loss: 0.3909 - val_acc: 0.9333
Epoch 4728/5000
50/50 [==============================] - 0s 3ms/step - loss: 5.6167e-06 - acc: 1.0000 - val_loss: 0.3922 - val_acc: 0.9333
Epoch 4729/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.5153e-07 - acc: 1.0000 - val_loss: 0.3922 - val_acc: 0.9333
Epoch 4730/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.5331e-06 - acc: 1.0000 - val_loss: 0.3919 - val_acc: 0.9333
Epoch 4731/5000
50/50 [==============================] - 0s 2ms/step - loss: 7.8560e-07 - acc: 1.0000 - val_loss: 0.3917 - val_acc: 0.9333
Epoch 4732/5000
50/50 [==============================] - 0s 2ms/step - loss: 7.9513e-07 - acc: 1.0000 - val_loss: 0.3916 - val_acc: 0.9333
Epoch 4733/5000
50/50 [====

Epoch 4785/5000
50/50 [==============================] - 0s 3ms/step - loss: 5.2478e-06 - acc: 1.0000 - val_loss: 0.3877 - val_acc: 0.9467
Epoch 4786/5000
50/50 [==============================] - 0s 3ms/step - loss: 6.2943e-07 - acc: 1.0000 - val_loss: 0.3877 - val_acc: 0.9467
Epoch 4787/5000
50/50 [==============================] - 0s 2ms/step - loss: 3.2045e-06 - acc: 1.0000 - val_loss: 0.3878 - val_acc: 0.9467
Epoch 4788/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.4687e-06 - acc: 1.0000 - val_loss: 0.3878 - val_acc: 0.9467
Epoch 4789/5000
50/50 [==============================] - 0s 2ms/step - loss: 4.1723e-07 - acc: 1.0000 - val_loss: 0.3878 - val_acc: 0.9467
Epoch 4790/5000
50/50 [==============================] - 0s 2ms/step - loss: 5.9009e-07 - acc: 1.0000 - val_loss: 0.3878 - val_acc: 0.9467
Epoch 4791/5000
50/50 [==============================] - 0s 3ms/step - loss: 5.9486e-07 - acc: 1.0000 - val_loss: 0.3878 - val_acc: 0.9467
Epoch 4792/5000
50/50 [====

Epoch 4844/5000
50/50 [==============================] - 0s 3ms/step - loss: 8.1063e-07 - acc: 1.0000 - val_loss: 0.3870 - val_acc: 0.9467
Epoch 4845/5000
50/50 [==============================] - 0s 3ms/step - loss: 3.4094e-07 - acc: 1.0000 - val_loss: 0.3871 - val_acc: 0.9467
Epoch 4846/5000
50/50 [==============================] - 0s 3ms/step - loss: 4.8757e-07 - acc: 1.0000 - val_loss: 0.3870 - val_acc: 0.9467
Epoch 4847/5000
50/50 [==============================] - 0s 3ms/step - loss: 6.1989e-07 - acc: 1.0000 - val_loss: 0.3871 - val_acc: 0.9467
Epoch 4848/5000
50/50 [==============================] - 0s 3ms/step - loss: 4.1366e-07 - acc: 1.0000 - val_loss: 0.3871 - val_acc: 0.9467
Epoch 4849/5000
50/50 [==============================] - 0s 3ms/step - loss: 7.5662e-06 - acc: 1.0000 - val_loss: 0.3865 - val_acc: 0.9467
Epoch 4850/5000
50/50 [==============================] - 0s 2ms/step - loss: 9.8944e-07 - acc: 1.0000 - val_loss: 0.3865 - val_acc: 0.9467
Epoch 4851/5000
50/50 [====

Epoch 4903/5000
50/50 [==============================] - 0s 3ms/step - loss: 3.4571e-07 - acc: 1.0000 - val_loss: 0.3874 - val_acc: 0.9400
Epoch 4904/5000
50/50 [==============================] - 0s 3ms/step - loss: 6.5123e-05 - acc: 1.0000 - val_loss: 0.3857 - val_acc: 0.9333
Epoch 4905/5000
50/50 [==============================] - 0s 3ms/step - loss: 5.0085e-06 - acc: 1.0000 - val_loss: 0.3846 - val_acc: 0.9333
Epoch 4906/5000
50/50 [==============================] - 0s 3ms/step - loss: 6.0916e-07 - acc: 1.0000 - val_loss: 0.3845 - val_acc: 0.9333
Epoch 4907/5000
50/50 [==============================] - 0s 3ms/step - loss: 7.7009e-07 - acc: 1.0000 - val_loss: 0.3846 - val_acc: 0.9333
Epoch 4908/5000
50/50 [==============================] - 0s 3ms/step - loss: 6.9380e-07 - acc: 1.0000 - val_loss: 0.3846 - val_acc: 0.9333
Epoch 4909/5000
50/50 [==============================] - 0s 3ms/step - loss: 4.1127e-07 - acc: 1.0000 - val_loss: 0.3845 - val_acc: 0.9333
Epoch 4910/5000
50/50 [====

Epoch 4962/5000
50/50 [==============================] - 0s 2ms/step - loss: 3.2663e-07 - acc: 1.0000 - val_loss: 0.3837 - val_acc: 0.9400
Epoch 4963/5000
50/50 [==============================] - 0s 3ms/step - loss: 6.9142e-07 - acc: 1.0000 - val_loss: 0.3837 - val_acc: 0.9400
Epoch 4964/5000
50/50 [==============================] - 0s 3ms/step - loss: 4.6730e-07 - acc: 1.0000 - val_loss: 0.3837 - val_acc: 0.9400
Epoch 4965/5000
50/50 [==============================] - 0s 3ms/step - loss: 8.1540e-07 - acc: 1.0000 - val_loss: 0.3836 - val_acc: 0.9400
Epoch 4966/5000
50/50 [==============================] - 0s 3ms/step - loss: 2.9445e-07 - acc: 1.0000 - val_loss: 0.3836 - val_acc: 0.9400
Epoch 4967/5000
50/50 [==============================] - 0s 3ms/step - loss: 1.4711e-06 - acc: 1.0000 - val_loss: 0.3835 - val_acc: 0.9400
Epoch 4968/5000
50/50 [==============================] - 0s 3ms/step - loss: 6.1989e-07 - acc: 1.0000 - val_loss: 0.3836 - val_acc: 0.9400
Epoch 4969/5000
50/50 [====

# Recorded results
## Adiac

|Run |Loss |Accuracy | Comment
|:---|:--- |:---     |:----------
|1   |0.005222544357037315  |0.6930946294608933 |Adadelta - default values
|2   |0.0018498263113997382 |0.7109974427601261 |
|3   |0.001620212956988372  |0.7212276216358176 |Adadelta parameters match Wang (2017)
|4   |0.0023210097823697976 |0.7289002559069172  |

## GunPoint

|Run |Loss |Accuracy | Comment
|:---|:--- |:---     |:----------
|1   | 1.466274483163943e-07  | 0.9400000035762787 |Adadelta parameters match Wang (2017)
|2   | 1.370907000364241e-07  | 0.9333333363135655  |
|3   | 1.3589860969887013e-07 | 0.9333333373069763  |

# Read results from file

In [14]:
read_log = pd.read_csv(logs_dir+'/'+fname+'/history.csv')
read_loss = read_log.loc[read_log['loss'].idxmin]['loss']
read_val_acc = read_log.loc[read_log['loss'].idxmin]['val_acc']
print('Read from file', logs_dir+'/'+fname+'/history.csv', '- loss and val_acc are')
print(read_loss, read_val_acc)

Read from file ../logs/2018-11-17T14:19/GunPoint/history.csv - loss and val_acc are
1.358986096988701e-07 0.9333333373069764
